In [1]:
# Code adapted from: https://www.analyticsvidhya.com/ + 
# blog/2021/12/text-classification-using-bert-and-tensorflow/

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Data
df = pd.read_csv("../TwitterData/prelim_dataset.csv")

# TODO: Text pre-processing?
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['Islamophobic?'], stratify=df['Islamophobic?'], random_state = 0)

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 32)
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
# Threshold
y_predicted = np.where(y_predicted > 0.5, 1, 0)

# Metrics
accuracy_score(y_test, y_predicted)
print("Macro:", f1_score(y_test, y_predicted, average='macro'))
print("Micro:", f1_score(y_test, y_predicted, average='micro'))

Epoch 1/2
19/19 [==============================] - 417s 18s/step - loss: 0.5678 - accuracy: 0.7905
Epoch 2/2
7/7 [==============================] - 107s 14s/step
Macro: 0.4515235457063712
Micro: 0.8232323232323232


In [4]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90       163
         1.0       0.00      0.00      0.00        35

    accuracy                           0.82       198
   macro avg       0.41      0.50      0.45       198
weighted avg       0.68      0.82      0.74       198



c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
